# Ejercicio 8 

Toma tu número $n$ de la lista publicada para este ejercicio. 

1. Pasa algunos tests de primalidad para ver si n es compuesto.
2. En caso que tu $n$ sea probable primo, factoriza $n+1$ encontrando certificados de primalidad para los factores mayores de $10000$.
3. Con $P = 1$, encuentra el menos $Q \ge 2$ tal que definan una sucesión de Lucas que certifique la primalidad de $n$. 

In [22]:
using Primes, Combinatorics

In [23]:
n = 2783584280956932824984484769172860189421

2783584280956932824984484769172860189421

## Apartado 1

Vamos a pasarle los test de Solovay-Strassen y de Miller-Rabin

In [24]:
function test_solovay_strassen(a, n, salida = false)
	if salida
		println("Test de Solovay Strassen para a = ", a, ", n = ", n)
	end

	powmod = powermod(a, div(n-1, 2), n)
	
	if powmod ∉ [1, n-1]
		if salida
			println("\ta^{(n-1)/2} mod n = ", powmod, ", que no es congruente con 1 o -1. Falla.")
		end
		return false
	end

	if salida
		println("\ta^{(n-1)/2} mod n = ", powmod, ". Jacobi = ", jacobisymbol(a, n), "\n")
	end

	return mod(jacobisymbol(a, n), n) == powmod
end

test_solovay_strassen (generic function with 2 methods)

In [25]:
primos = [2, 3, 5, 7, 9, 11, 13, 17]

8-element Vector{Int64}:
  2
  3
  5
  7
  9
 11
 13
 17

In [26]:
map(a -> test_solovay_strassen(a, n), primos)

8-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1
 1

In [45]:
# Julia utiliza por defecto el algoritmo de Miller-Rabin para probar si un número es primo, 
# siempre que este sea suficientemente grande (como es el caso).
# Mi función no era correcta del todo en los ejercicios anteriores, así que he optado por utilizar 
# la propia del lenguaje en este caso para asegurar la correctitud del ejercicio. 
# https://cutt.ly/aSvL5ap
isprime(big(n))

true

El resultado de estos tests nos dicen que nuestro $n$ es un posible primo. 

## Apartado 2

Como $n$ es un posible primo, vamos a factorizar $n+1$ usando el mismo método del ejercicio anterior:

In [28]:
function lucas_lehmer(n)
	factores = collect(keys(factor(n-1)))

	for a in 1:(n-1)

		if powermod(a, n-1, n) == 1
			resultado = map(q -> powermod(a, div(n-1, q), n), factores)
			if 1 ∉ resultado
				return true
			else
				posicion = first(findall(x -> x == 1, resultado))
			end
		end
	end

	return false
end

lucas_lehmer (generic function with 1 method)

In [29]:
function ρ_Polard(n, f, t = 1000, x0 = 2)
	# En caso en el que sea divisible por dos, es posible que falle, así que lo comprobamos a mano.
	if mod(n, 2) == 0
		return [[2, div(n, 2)], 0]
	end

	x = x0
	y = x
	i = 0

	while i < t
		i = i + 1
		x = mod(f(x), n)
		y = mod(f(f(y)), n)
		g = gcd(x - y, n)

		if 1 < g && g < n
			return [[g, div(n, g)], i]
		end
	end

	return []
end

function descomponer(n)
	por_descomponer = [n]
	irreducibles    = []
	
	f     = x -> x^2 + 1
	f_alt = x -> x^2 - 1
	
	i = 0

	while length(por_descomponer) > 0
		println("\nPor descomponer: $por_descomponer")
		num = pop!(por_descomponer)

		# Aplica el test de Miller Rabin primero. Si sale posible primo, saca 
		# un certificado de primalidad con Lucas Lehmer.
		if isprime(num) && lucas_lehmer(num)
			println("\t-> $num es primo")
			push!(irreducibles, num)
		else
			println("\t-> Descomponiendo $num")
			resultado = ρ_Polard(num, f, 3000)

			# Dado que ρ_Polard puede fallar, probamos con otra función alternativa. 
			# Por ejemplo, para 25, x -> x^2 + 1, no se devuelve nada. 
			
			if length(resultado) == 0
				resultado = ρ_Polard(num, x -> x^2 - 1, 4000)
			end
			
			i = i + resultado[2]
			por_descomponer = vcat(por_descomponer, resultado[1])
		end
	end

	return [sort(irreducibles), i]
end

descomponer (generic function with 1 method)

In [30]:
factores = descomponer(n+1)


Por descomponer: BigInt[2783584280956932824984484769172860189422]
	-> Descomponiendo 2783584280956932824984484769172860189422

Por descomponer: BigInt[2, 1391792140478466412492242384586430094711]
	-> Descomponiendo 1391792140478466412492242384586430094711

Por descomponer: BigInt[2, 3, 463930713492822137497414128195476698237]
	-> Descomponiendo 463930713492822137497414128195476698237

Por descomponer: BigInt[2, 3, 263, 1763995108337726758545300867663409499]
	-> Descomponiendo 1763995108337726758545300867663409499

Por descomponer: BigInt[2, 3, 263, 2237, 788553915215792024383236865294327]
	-> Descomponiendo 788553915215792024383236865294327

Por descomponer: BigInt[2, 3, 263, 2237, 487, 1619207218102242349862909374321]
	-> Descomponiendo 1619207218102242349862909374321

Por descomponer: BigInt[2, 3, 263, 2237, 487, 487, 3324860817458403182470039783]
	-> Descomponiendo 3324860817458403182470039783

Por descomponer: BigInt[2, 3, 263, 2237, 487, 487, 275131, 12084646286526793354693]
	-> 

2-element Vector{Any}:
    Any[2, 3, 263, 487, 487, 2237, 275131, 12084646286526793354693]
 823

In [31]:
factor(n+1)

2 * 3 * 263 * 487^2 * 2237 * 275131 * 12084646286526793354693

Tomamos ahora los factores mayores que $10000$

In [32]:
a_comprobar = filter(p -> p > 10000, factores[1])

2-element Vector{Any}:
                  275131
 12084646286526793354693

Le pasamos Lucas Lehmer a dichos factores, y obtenemos

In [33]:
map(p -> lucas_lehmer(p), a_comprobar)

2-element Vector{Bool}:
 1
 1

Así que todos nuestros factores son primos.

## Apartado 3

Solo nos queda buscar el menor natural $Q \ge 2$ tal que definan una sucesión de Lucas que certifique la primalidad de $n$. Para ello, usaremos las mismas funciones del ejercicio anterior.

In [34]:
function U_binario(P, Q, r, modulo = Inf)
	n      = reverse(digits(r, base = 2))
	k      = 0
	pareja::Tuple{BigInt, BigInt} = (0, 1)

	for e in n
		if e == 0
			pareja = (
                mod(2 * pareja[1] * pareja[2] - P * pareja[1]^2, modulo),
                mod(pareja[2]^2 - Q * pareja[1]^2, modulo)
            )
		else
            pareja = (
                mod(pareja[2]^2 - Q * pareja[1]^2, modulo),
                mod(P * pareja[2]^2 - 2 * Q * pareja[1] * pareja[2], modulo)
            ) 
        end
		k = k + 1
	end

	return (
		pareja, 
		2*pareja[2] - P*pareja[1]
	)
end

U_binario (generic function with 2 methods)

In [50]:
r, factores_r = n + 1, collect(keys(factor(r)))

(2783584280956932824984484769172860189422, BigInt[2, 3, 263, 487, 2237, 275131, 12084646286526793354693])

Procedemos a buscar nuestro $Q$:

In [51]:
P, Q = 1, 2

while  U_binario(P, Q, r, n)[1][1] != 0 && 
       0 ∉ map(p -> U_binario(P, Q, div(r, p), n)[1][1], factores_r)

    Q = Q + 1
end

Q

2

Verifiquemos que el resultado es correcto: Se tiene que $U_r = \dots$

In [53]:
U_binario(P, Q, r, n)[1][1]

0

Mientras que $U_{r/p},\ p \in \text{factores(} r\text{)}\dots$

In [39]:
map(p -> U_binario(P, Q, div(r, p), n)[1][1], factores_r)

7-element Vector{BigInt}:
 1164917580415388545044585558421187846643
 2657412299953816308871411867622873907412
 2607479893503099484201978595521426341475
 1573461913841534528242027443422651522291
 1345389753832447580730736572999126848012
  128397738086795830162502257971228794870
 2582704601212336087373341880436630298442

Así que $Q = 2$